In [ ]:
import os
from glob import glob
from imagedaemon import registry
# let's try to run the winter calibration pipeline
from imagedaemon.utils.paths import RAW_DATA_DIR
from imagedaemon.utils.image import Image
from imagedaemon.utils.wcs_utils import pix2sky



available_cameras = registry.available()
print("Registered cameras:", available_cameras)

print_cameras_found = False
if print_cameras_found:
    # check the camera parameters
    for camname in available_cameras:
        camera = registry.get(camname)
        print("camera:")
        for entry in camera.meta:
            print(f"\t {entry}")

PROJECT_ROOT = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/winter-image-daemon/imagedaemon
CONFIG_DIR = /Users/nlourie/Desktop/Work/MIT/WINTER/GIT/winter-image-daemon/imagedaemon/imagedaemon/config
Registered cameras: ['qcmos', 'summer-ccd', 'winter']
camera:
	 ('name', 'qcmos')
	 ('pixel_scale', 0.157)
	 ('cal_steps', {'dark': False, 'lab_flat': False, 'dither_flat': False, 'sky_flat': False, 'remove_horizontal_stripes': False, 'mask': False, 'mask_hot_pixels': True, 'replace_nans_with_median': True})
	 ('focus_cal_steps', {'dark': True, 'lab_flat': False, 'dither_flat': True, 'sky_flat': False, 'remove_horizontal_stripes': False, 'mask': False, 'mask_hot_pixels': False, 'replace_nans_with_median': True})
	 ('scale_margin', 0.05)
	 ('hot_pixel_threshold', 200)
	 ('focus_addrs', [])
camera:
	 ('name', 'summer-ccd')
	 ('pixel_scale', 0.466)
	 ('cal_steps', {'dark': False, 'lab_flat': False, 'dither_flat': False, 'sky_flat': False, 'remove_horizontal_stripes': False, 'mask': False, 'mask_

In [ ]:
# try to set up the focus
summerccd = registry.get("summer-ccd")

focus_dir = os.path.join(RAW_DATA_DIR, "summer-ccd", "focus")
focus_images = sorted(glob(os.path.join(focus_dir, "*.fits")))

# plot one
img = Image(focus_images[0])
img.plot_image(title = f"Focus Image: {img.filename}\nFocus Position: {img.header['FOCPOS']:.0f}",)


print(f"Focus images: {focus_images}")
print(f"type(focus_images): {type(focus_images)}")
res = summerccd._load_focus_images(focus_images)
print(f"Focus images loaded: {res}")
# calibrate all the focus images
focus_calibrated = summerccd.calibrate_for_focus(
    image_paths = focus_images,
    out_dir = os.path.join(os.getenv("HOME"), "data", "tmp"),
)

print(f"Focus calibrated images: {focus_calibrated}")

# now try for a focus loop

results = summerccd.run_focus_loop(image_list = focus_images,
                                   addrs = None,
                                   output_dir = None)

# plot the first one
#focus_calibrated[0].plot_image(f"First Focus Calibrated Image: {focus_calibrated[0].filename}",)



In [ ]:
# now run astrometry on a summer-ccd image

# try again with a full summer-ccd image
summerccd = registry.get("summer-ccd")

image_dir = os.path.join(RAW_DATA_DIR, "summer-ccd", "focus")
science_images = sorted(glob(os.path.join(image_dir, "*.fits")))
science_image = science_images[0]
print(f"Science image: {science_image}")

# plot the image just for fun
image = Image(science_image)
image.plot_image(title = f"Science Image: {image.filename}",)

# run the full astrometric pipeline
output_dir = os.path.join(os.path.join(os.getenv("HOME"), "data", "tmp"))

# background images
background_images = science_images


# this image has no RA/Dec in the header, so we need to set it
ra_guess = None
dec_guess = None

# calibrate the image
calimg = summerccd.calibrate_image(science_image = science_image, 
                                 background_image_list = background_images)

# try to save the calibrated image
# plot the image
calimg.plot_image(title = f"Calibrated Image: {calimg.filename}",)

info = summerccd.get_astrometric_solution(science_image = science_image,
                                      background_image_list = None,
                                      output_dir = output_dir,
                                      ra = None,
                                      dec = None,
    )
wcs = info["wcs"]
# the image is 4096 x 2304
wx = 2048
wy = 2049
# let's try to get the WCS for a pixel in the center of the image
x_pix, y_pix = wx // 2, wy // 2
sky_pos = pix2sky(x_pix, y_pix, wcs)
print(
    f"Coord conversion: (x, y) = ({x_pix}, {y_pix}) -> (RA, Dec) = ({sky_pos.ra:.1f}, {sky_pos.dec:.1f}) deg"
)

In [ ]:
# now try for a full focus loop


In [ ]:
# winter focus loop

winter_focus_dir = os.path.join(RAW_DATA_DIR, "winter", "focus")
winter_focus_images = sorted(glob(os.path.join(winter_focus_dir, "*.fits")))
from imagedaemon.cameras.winter.winter_image import WinterImage

# plot one
img = WinterImage(winter_focus_images[0])
#img.plot_image(title = f"Focus Image: {img.filename}\nFocus Position: {img.top_level_header['FOCPOS']:.0f}",)
# calibrate all the focus images
winter = registry.get("winter")
winter_focus_calibrated = winter.calibrate_for_focus(
    image_paths = winter_focus_images,
    out_dir = os.path.join(os.getenv("HOME"), "data", "tmp"),
)
print(f"Winter Focus calibrated images: {winter_focus_calibrated}")
# plot the first one
winter_focus_calibrated[0].plot_image(f"First Winter Focus Calibrated Image: {winter_focus_calibrated[0].filename}",)

In [ ]:
summerccd = registry.get("summer-ccd")

print(f"science_image type: {type(science_image)}")